This notebook is used to test the idea of uing a convex combiantion of Runge Kutta methods to get the final result.

For this approach the implicit solver is adventagious because at least the backward euler method yields to an positive result.
This enshures that at least one point of the convex combination is positive.

The idea is to have multiple $b_i$ for $d$ methods, that all can be computed with one $A$. 

This has the advantage of enshuring the solution is does not get worse than the solution of the backward euler method. (A good example for comparison is the Diffusion with an Dirca as initial  condition)

The optimisation problem is changed to:

$$u_{n+1}=u_n+\Delta K \vec{b} \geq 0$$

with

$$ \vec{b} = \sum_i^d a_i \vec{b_i}$$

$$ \forall_{i \in \{1, \cdots d \}} 0 \leq a_1 \leq 1$$

and the objective

$$ min \left(\sum_i^d w_i a_i \right) $$

with some weighting factors $w_1,\cdots,w_d$. The weighting factors determine which is the prefered b to use.

Because we want that the optimal soultion is the origianl $b$ the weight $w_0$ corresponding to $b_{orig}$ should be the smalest $w$. We could just set this $w$ to $0$.
For implicit methods with $s \geq p$ we would still like to have some degrees of fredum for the highest order. For this we have to intoduce at least one $b$ that complies with the order Condition for the Order and $b \neq b_{orig}$. 

If we reduce the Order we can intoduce new $b$ that do not complie the Order Conditons (This is an easy task for the first Order because we have to choose a method that ensures positifity but is not so clear cut for the intermediate Orders)

What would be good objectives for choosing the $b$ od Order $> 1$?:
Small negative b?
Small error coefficents?


We also have to make sure that all used $b$ define A-Stable methods, because this ensures that the final method is alos A-Stable

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg


#Diagonally Implicit methods:
BE = rk.loadRKM('BE').__num__()
SDIRK23 = rk.loadRKM('SDIRK23').__num__()
SDIRK34 = rk.loadRKM('SDIRK34').__num__()
SDIRK54 = rk.loadRKM('SDIRK54').__num__()
TR_BDF2 = rk.loadRKM('TR-BDF2').__num__()


be = rk.loadRKM('BE').__num__()

#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()


from OrderCondition import *
from RKimple import *
import utils 
import scipy.linalg 

In [ ]:
print(ex2)
print(ex3)

# Adding the embeded Methods

We want to get new methods that complie with the Order Conditions. Thsi can be done by getting the vectors that span the Kernel of the Order Condition MAtrix $Q$. This vectors can then be weighted and added to the original $b$. By this the Order Conditions are still satisfied. 

As last step we test if the gerneated method is still A-Stable 


Note on the Implementation:
We use the Attribute A-hat of the rkm class to store the embeded emthods. We give it as a dict for the different Orders.

In [ ]:
Q,rhs = OrderCond(ex3.A,ex3.c,order = 3)
scipy.linalg.null_space(Q)

In [ ]:
#thsi is probably not the optimal way but it still gives all possible options
span = scipy.linalg.null_space(Q)
scale = np.array([[2,2],[2.5,2.5],[0.5,0.5],[1.2,0.2]]) #one set od scales that lead to A-Stable methods
scale = np.array([[0.1,0.1],[0.1,0.1],[0.03,0.03],[0.1,0.05]]) #another set that shows less distortion 
ex3 = rk.extrap(3,'implicit euler').__num__()
b_orig = ex3.b
b_hat_3 = []
for i in range(span.shape[1]):
    for s in [-1,+1]:
        ex3.b = b_orig+ scale[i][int(0.5+0.5*s)]*s*span[:,i]
        b_hat_3.append(ex3.b)
        ex3.plot_stability_region();



In [ ]:
#as the seccond order method we try projecting the first oder method on the order conditions 
#but we will provbably need something better here
db = np.array([0,0,0,0.3,0.3,0.3]) -ex3.b
display(db)
Q,rhs = OrderCond(ex3.A,ex3.c,order = 2)
S = scipy.linalg.null_space(Q)
#construct proj. Matrix
proj = np.zeros([S.shape[0],S.shape[0]])
for i in range(S.shape[1]):
    s = S[:,0]
    s.shape = (len(s),1)
    proj += s@s.T
display(proj)
b_proj = proj@db + ex3.b
print('Test if Order condition are still mett')
display(Q@b_proj-rhs)
print('New b of second Order')
display(b_proj)

ex3.b = b_proj
ex3.plot_stability_region();
#apparenty that does not work...

In [ ]:
#some guess instead [-1/2,-1,-1/2,1,1,1]
ex3.b = np.array([-1/2,-1,-1/2,1,1,1])
ex3.plot_stability_region();

b_hat_2 = [np.array([-1/2,-1,-1/2,1,1,1])]
#b_hat_2 = [np.array([0.5,-2.,-2.,1.5,1.5,1.5])] #not real new set 

In [ ]:
#The first Order method that ensures posifitifity
ex3.b = np.array([0,0,0,1/3,1/3,1/3])
ex3.plot_stability_region();

b_hat_1 = [np.array([0,0,0,1/3,1/3,1/3])]

In [ ]:
ex3 = rk.extrap(3,'implicit euler').__num__()
ex3.b_hat = {3:b_hat_3,2:b_hat_2,1:b_hat_1}

#construct one method that only has the implicit euler as another option
#b_hat_orig = [ex3.b]
#ex3.b_hat = {3:b_hat_orig,2:b_hat_orig,1:b_hat_1}

## Weights

It remains the question on what weights to use. 
We already set $w_0 = 0$ for the other weights we use $w_i = \frac{1}{Order\{b_i\}}$

In [ ]:
N=100 #interestingly the behavior depends heaviliy on the number of points
x = np.linspace(0,1,N)
dx = x[1]-x[0]
u0 = np.zeros_like(x)
u0[int(N/2)] = 1
#u0[int(N/2)-2:int(N/2)+2] = 1
#dt = 0.7*dx**2
dt = 0.0007847599703514606
#dt = 7e-5

A_heat = 1/dx**2 * (-2*np.diag(np.ones(N))+np.diag(np.ones(N-1),-1)+np.diag(np.ones(N-1),1))


#t, u, b = RK_variable_b_implicit(ex8,dt,f_heat,w0=u0,t_final=1.1,solver_eqs =solver_nonlinear_arg,
#                                 b_fixed=True,solver=cp.SCS)



solver = Solver(rkm = ex3,
               dt = dt,
               t_final = 0.01,
               b_fixed=False,
               tol_neg=1e-8,
               tol_change = np.inf,
               p = [3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               solver_eqs=solver_Matrix,
               convex = False,
               LP_opts = {'reduce':True})

problem_ADR = Problem(f=A_heat,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)


status,t,u,b,KK = RK_integrate(solver=solver,problem=problem_ADR,verbose=True,dumpK=True)

t_ref = np.array(t)
u_ref = np.array(u).T
b_ref = np.array(b).T
utils.show_status(status)

#t, u, b = RK_convex_implicit(ex3,B,w,dt,A_heat,w0=u0,t_final=0.01,solver_eqs =solver_Matrix,
#                                 b_fixed=False,solver=cp.SCS,fallback = True)

#print('ref:')

#t_ref, u_ref, b_ref = RK_variable_b_implicit(ex3,dt,A_heat,w0=u0,t_final=0.01,solver_eqs =solver_Matrix,
#                                 b_fixed=True,solver=cp.SCS,fallback = True)


u_ex = scipy.linalg.expm(dt*A_heat)@u0

In [ ]:
solver = Solver(rkm = ex3,
               dt = dt,
               t_final = 0.01,
               b_fixed=False,
               tol_neg=1e-8,
               tol_change = np.inf,
               p = [3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               solver_eqs=solver_Matrix,
               convex=True,
               LP_opts = {'reduce':True})

status,t,u,b,KK = RK_integrate(solver=solver,problem=problem_ADR,verbose=True,dumpK=True)

t_con = np.array(t)
u_con = np.array(u).T
b_con = np.array(b).T
utils.show_status(status)

In [ ]:
plt.pcolor(u_con[:,:10])
plt.colorbar()

In [ ]:
plt.pcolor(u_ref[:,:10])
plt.colorbar()

In [ ]:
plt.plot(t[1:],b_con.T[1:,:]);

In [ ]:
plt.plot(t[1:],b_ref.T[1:,:]);

In [ ]:
n=1
plt.plot(u_con[:,n],label='convex')
plt.plot(u_ref[:,n],label='ref')
plt.plot(u_ex,label='ex')
plt.legend()

In [ ]:
for i in range(6):
    plt.plot(KK[1][:,i],'-o',label=str(i))
#plt.ylim([-0.1,0.1])
plt.plot(KK[1]@ex3.b,'-o',label='combined')
plt.grid()
plt.legend()

In [ ]:
plt.figure()
#plot the differnet options for the first step
plt.plot(u0 + dt*KK[1]@ex3.b,'-x',label='regular')
plt.plot(u0 + dt*KK[1]@b_hat_3[1],'-x',label='b_hat3')
plt.plot(u0 + dt*KK[1]@b_hat_2[0],'-x',label='b_hat2')
plt.plot(u0 + dt*KK[1]@b_hat_1[0],'-x',label='b_hat1')
plt.grid()
plt.legend()


plt.figure()
for i in range(len(b_hat_3)):
    plt.plot(u0 + dt*KK[1]@b_hat_3[i],'-x',label=str(i))
plt.legend()

## Convergence

In [ ]:
from scipy.linalg import expm


N=100
x = np.linspace(0,1,N)
dx = x[1]-x[0]
u0 = np.zeros_like(x)
u0[int(N/2)] = 1
#dt = 0.7*dx**2
dt = 0.001

A_heat = 1/dx**2 * (-2*np.diag(np.ones(N))+np.diag(np.ones(N-1),-1)+np.diag(np.ones(N-1),1))



#dt = np.array([0.1,0.01,0.001,0.0001])
dts = np.logspace(-5,-2,num=20)

t_end = 0.01

reference = np.zeros([len(u0),len(dts)])
for i in range(len(dts)):
    reference[:,i]= expm(t_end*A_heat)@u0
    

problem_Heat = Problem(f=A_heat,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

solver_be = Solver(rkm = be,
               dt = dt,
               t_final = t_end,
               b_fixed=True,
               tol_neg=1e-8,
               tol_change = 5,
               p = [3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               solver_eqs=solver_Matrix,
               LP_opts = {'reduce':True})

solver_ex3_fix = Solver(rkm = ex3,
               dt = dt,
               t_final = t_end,
               b_fixed=True,
               tol_neg=1e-8,
               tol_change = 5,
               p = [3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               solver_eqs=solver_Matrix,
               LP_opts = {'reduce':True})

solver_ex3_adp = Solver(rkm = ex3,
               dt = dt,
               t_final = t_end,
               b_fixed=False,
               tol_neg=1e-8,
               tol_change = 5,
               p = [3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               solver_eqs=solver_Matrix,
               convex=False,
               LP_opts = {'reduce':True,'verbose':False})


solver_ex3_cvx = Solver(rkm = ex3,
               dt = dt,
               t_final = t_end,
               b_fixed=False,
               tol_neg=1e-8,
               tol_change = 5,
               p = [3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               solver_eqs=solver_Matrix,
               convex=True,
               LP_opts = {'reduce':True,'verbose':False})

print('BE')
sol_be,err_be,change_be = utils.plot_convergence(problem_Heat,solver_be,dts,reference,error='rel',step = -1,
                              Params=dict())

print('ex3')
sol_ex3,err_ex3,change_ex3 = utils.plot_convergence(problem_Heat,solver_ex3_fix,dts,reference,error='rel',step = -1,
                              Params=dict())

print('ex3_convex')
sol_ex3c,err_ex3c,change_ex3c = utils.plot_convergence(problem_Heat,solver_ex3_cvx,dts,reference,error='rel',step = -1,
                              Params=dict())

print('ex3_adapted')
sol_ex3a,err_ex3a,change_ex3a = utils.plot_convergence(problem_Heat,solver_ex3_adp,dts,reference,error='rel',step = -1,
                              Params=dict())



In [ ]:
plt.loglog(dts,err_be,'C1',label = 'BE')
plt.loglog(dts,err_ex3,'C2',label = 'ex3')
plt.loglog(dts,err_ex3a,'C3',label = 'ex3a')
plt.loglog(dts,err_ex3c,'C4',label = 'ex3c')

plt.plot(dts[False==change_be],err_be[False==change_be],'oC1')
plt.plot(dts[False==change_ex3],err_ex3[False==change_ex3],'oC2')
plt.plot(dts[False==change_ex3a],err_ex3a[False==change_ex3a],'oC3')
plt.plot(dts[True==change_ex3a],err_ex3a[True==change_ex3a],'xC3')
plt.plot(dts[False==change_ex3c],err_ex3c[False==change_ex3c],'oC4')
plt.plot(dts[True==change_ex3c],err_ex3c[True==change_ex3c],'xC4')

plt.legend()
plt.grid()

"""Note to plot: For dt nearly equal to the endtime the results may be skewed 
because one large step and another smal step were taken to compute the solution """

The convergence shows interesting behavior if the $b$ is constrained to a convex combination. 
If the $b$ can be chosen freely the error is usually equal or smaler than the error of the unaltered method. 
If the $b$ is constrained there is some kind of tiping point for that the error suddenly gets larger than the original error.... Maybe because the order was dropped....
Apparently it is a more demanding problem to choose the embedded methods, in ways that suit to the problem

In [ ]:
rkm = ex3
B = [[rkm.b]]
w = [0]
for order in range(rkm.p,0,-1):
    B.append(rkm.b_hat[order])
    w.extend([1/order]*len(rkm.b_hat[order]))
                
B = np.concatenate(B).T
n = np.arange(0,B.shape[1]) #only for plotting
plt.plot(n,B.T)

In [ ]:
print(b_hat_3)

In [ ]:
b_hat_3 = [np.array([ 0.87332328, -3.35313268, -2.81672679,  2.00742476,  2.51484951,1.77426192]), 
 np.array([ 0.12667672, -0.64686732, -1.18327321,  0.99257524,  0.48515049,1.22573808]), 
 np.array([ 0.75452072, -1.34140466, -3.34738056,  1.25302675,  1.00605349,2.67518426]), 
 np.array([ 0.24547928, -2.65859534, -0.65261944,  1.74697325,  1.99394651,0.32481574])]

In [ ]:
Q,rhs = OrderCond(ex3.A,ex3.c,order=3)

for i in range(len(b_hat_3)):
    ex3.b = b_hat_3[i]
    ex3.plot_stability_region()
    print(Q@ex3.b-rhs)

# Robertson

In [ ]:
#Robertson test problem, stiff
def f_robertson(t,u):
    a = 0.3
    du = np.zeros(3)
    du[0] = 1e4 *u[1]*u[2] - 0.04*u[0]
    du[1] = 0.04 *u[0] - 1e4*u[1]*u[2] - 3e7*u[1]**2
    du[2] = 3e7*u[1]**2
    return du

u0 = np.array([1.,0.,0.])
    
#    t,u,b = RK_variable_b(ssp104,1e-6,f_robertson,w0=u0,t_final=1,b_fixed=True)"

In [ ]:
dt_start = 1e-3

#here a smal hack to make it compatible with the stepsize contol code
ex3.bhat = ex3.b


solver = Solver(rkm = ex3,
               dt = dt_start,
               t_final = 1e12,
               b_fixed=False,
               tol_neg=1e-4,
               tol_change = 5,
               p = [4,3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               solver_eqs=solver_nonlinear_arg,
               convex=True,
               fail_on_requect=False,
               LP_opts = {'verbose_LP':False})

problem_robertson = Problem(f=f_robertson,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

def stepsize(stepsize_control,dt_old,error,change,success,tol_met):
    return dt_old*10

control = StepsizeControl(dt_min = 0,dt_max = np.infty,a_tol = 0.001,r_tol=0.001,f = stepsize
                          ,tol_reqect = 0.002)



#t,u,b,KK = RK_variable_b(ssp104,dt,f_prod,u0,t_final=5,b_fixed=False,dumpK=True)
status,t,u,b,KK = RK_integrate(solver=solver,problem=problem_robertson,stepsize_control=control,
                               verbose=True,dumpK=True)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
plt.semilogx(t,u.T[:,0])
plt.semilogx(t,1e4*u.T[:,1])
plt.semilogx(t,u.T[:,2])

In [ ]:
plt.plot(b.T)

# Additional Code

In [ ]:
#Plot stability region of first Step
ex3_ = rk.extrap(3,'implicit euler').__num__()
ex3_.b = b_ref[:,1]
ex3_.plot_stability_region();